This code uses the blackboard API to create gradebook columns from the syllabus

In [1]:
!pip install -q --user bbrest

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
from bbrest import BbRest
import pandas as pd
import requests
import datetime

In [3]:
def to_utc_in_isoformat(date):
    m, d, y = date.split("/")
    localtime = datetime.datetime(int(y), int(m), int(d), 23, 59, 0)
    utctime = localtime.astimezone(datetime.timezone.utc)
    return utctime.isoformat()


assert to_utc_in_isoformat("12/1/2023") == "2023-12-02T04:59:00+00:00"

In [4]:
response = requests.get("http://ist256.com/syllabus/", headers={
        "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"
    }
)
assignments = pd.read_html(response.text)[-1]

/tmp/ipykernel_232/2711504448.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  assignments = pd.read_html(response.text)[-1]


In [5]:
appkey = "89abd9f0-2aa5-4c10-8738-6394077e8b9d"
secret = "38kGcvte5VswC6nfvDcCDASf74RHEEO9"
appid = "8c67899c-5ded-4ba3-8c9e-f395bfa8d52e"
url = "https://blackboard.syracuse.edu"
cid = "17008.1251m" # course ID TODO set to current course id
bb = BbRest(appkey, secret, url)
courseinfo = bb.GetCourse(courseId=cid)
courseinfo.json()

{'id': '_530562_1',
 'uuid': 'b306f4127e9f4ae58f5819643a05bcf6',
 'externalId': '17008.1251m',
 'dataSourceId': '_2_1',
 'courseId': '17008.1251m',
 'name': 'IST.256.MERGED.FALL24.Intro to Python for Info Prof',
 'created': '2024-08-01T18:28:56.090Z',
 'modified': '2024-08-06T15:09:32.668Z',
 'organization': False,
 'ultraStatus': 'Ultra',
 'allowGuests': False,
 'allowObservers': True,
 'closedComplete': False,
 'termId': '_278_1',
 'availability': {'available': 'No',
  'duration': {'type': 'DateRange',
   'start': '2024-07-31T04:00:00.000Z',
   'end': '2025-01-31T05:00:00.000Z'}},
 'enrollment': {'type': 'InstructorLed'},
 'locale': {'force': False},
 'hasChildren': True,
 'externalAccessUrl': 'https://blackboard.syracuse.edu/ultra/courses/_530562_1/outline'}

In [6]:
response = bb.GetGradeColumns(courseId=cid)
gradebook_cols = response.json()

In [7]:
for a in assignments.to_records():
    for g in gradebook_cols['results']:
        if a['Gradebook'] == g['name']:
            g['score']['possible'] = float(a['Points'])
            old = g['grading']['due']
            g['grading']['due'] = to_utc_in_isoformat(a['Date Due'])
            print(f"Updating {g['name']} FROM:{old} TO:{g['grading']['due']} |points={g['score']['possible']}")
            bb.UpdateGradeColumn(courseId=cid, columnId=g['id'], payload=g)
            break

Updating L01 FROM:2024-01-24T04:59:00.000Z TO:2024-08-28T03:59:00+00:00 |points=3.0
Updating S01 FROM:2024-01-25T04:59:00.000Z TO:2024-08-29T03:59:00+00:00 |points=3.0
Updating H01 FROM:2024-01-27T04:59:00.000Z TO:2024-08-31T03:59:00+00:00 |points=3.0
Updating L02 FROM:2024-01-31T04:59:00.000Z TO:2024-09-04T03:59:00+00:00 |points=3.0
Updating S02 FROM:2024-02-01T04:59:00.000Z TO:2024-09-05T03:59:00+00:00 |points=3.0
Updating H02 FROM:2024-02-03T04:59:00.000Z TO:2024-09-07T03:59:00+00:00 |points=3.0
Updating L03 FROM:2024-02-07T04:59:00.000Z TO:2024-09-11T03:59:00+00:00 |points=3.0
Updating S03 FROM:2024-02-08T04:59:00.000Z TO:2024-09-12T03:59:00+00:00 |points=3.0
Updating H03 FROM:2024-02-10T04:59:00.000Z TO:2024-09-14T03:59:00+00:00 |points=3.0
Updating L04 FROM:2024-02-13T23:59:00.000Z TO:2024-09-18T03:59:00+00:00 |points=3.0
Updating S04 FROM:2024-02-14T23:59:00.000Z TO:2024-09-19T03:59:00+00:00 |points=3.0
Updating E1 FROM:2024-02-14T23:59:00.000Z TO:2024-09-19T03:59:00+00:00 |poin

In [10]:
g

{'id': '_2736325_1',
 'name': 'E2',
 'created': '2023-08-11T18:42:22.106Z',
 'score': {'possible': 45.0},
 'availability': {'available': 'Yes'},
 'grading': {'type': 'Manual',
  'due': '2023-12-12T04:59:00.000Z',
  'schemaId': '_2105660_1'},
 'gradebookCategoryId': '_4694889_1',
 'includeInCalculations': True,
 'showStatisticsToStudents': False}

In [ ]:
# OLD
def replace_date(date, timestamp):
    m, d, y = date.split("/")
    new_date = f"{int(y):04d}-{int(m):02d}-{int(d):02d}"
    time = timestamp.split("T")[-1]
    new_timestamp = f"{new_date}T23:59:00"
    return new_timestamp


assert replace_date("1/30/2023","1999-12-12T12:00") == '2023-01-30T12:00'